In [1]:
from keras.applications import VGG16

In [2]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [5]:
original_dataset_dir = 'F:\\study\\ml\\DataSet\\dogs_vs_cats\original'
base_dir = 'F:\study\ml\DataSet\dogs_vs_cats\cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [6]:
datagen = ImageDataGenerator(rescale=1. / 255)
batch_size = 20

In [9]:
def extract_feature(directory, sample_count):
    feats = np.zeros((sample_count, 4, 4, 512))
    labels = np.zeros(sample_count)

    data_generator = datagen.flow_from_directory(directory,
                                                 target_size=(150, 150),
                                                 batch_size=batch_size,
                                                 class_mode='binary')
    i = 0
    for input_batch, batch_label in data_generator:
        feat_batch = conv_base.predict(input_batch)
        feats[i * batch_size:(i + 1) * batch_size] = feat_batch
        labels[i * batch_size:(i + 1) * batch_size] = batch_label

        i += 1
        if i * batch_size >= sample_count:
            break
    return feats, labels

In [10]:
train_features, train_labels = extract_feature(train_dir, 2000)
validation_features, validation_labels = extract_feature(validation_dir, 1000)
test_features, test_labels = extract_feature(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [61]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [63]:
train_features = np.reshape(train_features, (2000, 4 , 4 , 512))
validation_features = np.reshape(validation_features, (1000, 4 , 4 , 512))
test_features = np.reshape(test_features, (1000, 4 , 4 , 512))

In [54]:
train_features.shape

(2000, 4, 4, 512)

In [14]:
from keras import models, layers, optimizers

In [82]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))


model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [91]:
train_features = np.reshape(train_features, (2000, 4 , 4 , 512))
validation_features = np.reshape(validation_features, (1000, 4 , 4 , 512))
test_features = np.reshape(test_features, (1000, 4 , 4 , 512))


model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(4,4,512)))
# model.add(layers.Dense(256, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [92]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

In [72]:
train_features.shape

(2000, 4, 4, 512)

In [73]:
train_labels.shape

(2000,)

In [93]:
history = model.fit(train_features,
                    train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 561us/step - loss: 0.7200 - acc: 0.4940 - val_loss: 0.6693 - val_acc: 0.6180
Epoch 2/30
2000/2000 [==============================] - 1s 406us/step - loss: 0.6758 - acc: 0.5790 - val_loss: 0.6321 - val_acc: 0.7400
Epoch 3/30
2000/2000 [==============================] - 1s 412us/step - loss: 0.6337 - acc: 0.6470 - val_loss: 0.5966 - val_acc: 0.7820
Epoch 4/30
2000/2000 [==============================] - 1s 412us/step - loss: 0.5891 - acc: 0.7130 - val_loss: 0.5618 - val_acc: 0.8060
Epoch 5/30
2000/2000 [==============================] - 1s 417us/step - loss: 0.5537 - acc: 0.7485 - val_loss: 0.5274 - val_acc: 0.8260
Epoch 6/30
2000/2000 [==============================] - 1s 413us/step - loss: 0.5163 - acc: 0.7775 - val_loss: 0.4961 - val_acc: 0.8360
Epoch 7/30
2000/2000 [==============================] - 1s 408us/step - loss: 0.4895 - acc: 0.7900 - val_loss: 0.4689 - val_acc: 0.8330
